# Synthetic data preprocessing

In [ ]:
import json
import pandas as pd
import re
import tiktoken

In [ ]:
# Data file folder and name
foldername = ""
filename = ""

output_filename = ""

In [ ]:
# Data and labels
synth_data = pd.read_csv(foldername + filename, sep="\t")

label_names = synth_data['label'].unique().tolist()
label_names.remove('O')

In [ ]:
# Prompts
system_content = "This model extracts entities from text, returning JSON-formatted output for tags " + ", ".join(label_names) + "."
instruction = "Extract entities " + ", ".join(label_names) + " from the following text and return the output in JSON format. "

# Context limit
token_limit = 4063
encoding = tiktoken.get_encoding("cl100k_base")

In [ ]:
def synth_data_to_pairs(foldername, filename, label_names):
    '''
    Function for formatting texts and annotations from input file.
    Annotations are formatted as a dictionary for both labels and indexes.

    The input file consists of multiple annotated texts. 
    The texts are split so the named entities are separate from the other text. 
    Each text begins with metadata, which is removed during processing.
    The texts are separated by empty rows.
    '''
    with open(foldername + filename) as f:
        synth_data = f.read().split("\n")

    texts = []
    annotations = []
    annotations_index = []
    
    current_text = []
    current_labels = {l: [] for l in label_names}  
    current_labels_index = {l: [] for l in label_names}
    
    start_adding = False
    current_index = 0

    # Each row consists of two parts: text and the corresponding label
    for line in synth_data:

        # Splitting the text and label 
        parts = line.split("\t")

        if len(parts) == 2:
            
            text, label = line.split("\t")
    
            if start_adding:
                # Named entity
                if label in label_names:
                    current_labels[label].append(text)
                    
                    label_index_tuple = (current_index, current_index + len(text))
                    current_labels_index[label].append(label_index_tuple)

                # Adding the part to the text
                current_text.append(text)
                current_index += len(text) + 1

            # The true text begins after the diagnosis code
            if re.match(r'\D\d\d-\d\d?', text):
                start_adding = True
                current_index = 0
    
        else:
            # The current text has ended and the data is added to lists
            if len(current_text) > 0:
                total_text = " ".join(current_text)
                texts.append(total_text)
                annotations.append(current_labels)
                annotations_index.append(current_labels_index)

            # New text
            current_text = []
            current_labels = {l: [] for l in label_names} 
            current_labels_index = {l: [] for l in label_names}
            start_adding = False
            current_index = 0

    return texts, annotations, annotations_index

In [ ]:
def format_messages(text, ann, ann_index):
    '''
    Function for formatting example messages.
    '''
    messages = []

    # System prompt
    sys = {}
    sys["role"] = "system"
    sys["content"] = system_content

    # Prompt and text
    user = {}
    user["role"] = "user"
    user["content"] = instruction + text

    # Labels
    ass = {}
    ass["role"] = "assistant"
    ass["content"] = ann

    # Labels by index
    ass_index = {}
    ass_index["role"] = "annotations"
    ass_index["content"] = ann_index

    messages.append(sys)
    messages.append(user)
    messages.append(ass)
    messages.append(ass_index)

    example = {}
    example["messages"] = messages

    return example

In [ ]:
def example_within_limit(example):
    '''
    Function to ensure that the example message is within the model's context limit.
    '''
    messages = example["messages"]

    total_tokens = 0
    
    for message in messages:
        if message["role"] != "annotations": # Excluding the annotation indexes, as they would not be included in the fine-tuning examples.
            total_tokens += len(encoding.encode(str(message["content"])))

    if total_tokens <= token_limit:
        return True

    return False

In [ ]:
def write_examples_to_file(filename, examples):
    '''
    Function for writing the formatted examples into a JSONL-file.
    '''
    with open(filename, "w", encoding='utf-8') as file:
        for example in examples:
            json.dump(example, file, ensure_ascii=False)
            file.write('\n')

In [ ]:
# Formatting text and annotations
texts, annotations, annotations_index = synth_data_to_pairs(foldername, filename, label_names)

# Creating the example messages
examples = []

if len(texts) == len(annotations):
    for i in range(len(texts)):
        example = format_messages(texts[i], annotations[i], annotations_index[i])

        if example_within_limit(example):
            examples.append(example)

In [ ]:
# Writing the formatted examples into a file
write_examples_to_file(output_filename, examples)